# LangChain Chat

## Setup

In [ ]:
# !pip install langchain langchain_community==0.0.36 langchain_chroma
# !pip install langchain-core
# !pip install bs4
# !pip install -qU langchain-groq
# !pip install python-dotenv

In [1]:
# !pip freeze > requirements.txt

In [11]:
from dotenv import load_dotenv
import os

load_dotenv()

hugging_api_key = os.getenv('HUGGING_FACE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

## Document Loading

In [12]:
from langchain_community.document_loaders import BSHTMLLoader

file_path = "./AkrabElikApp.html"

loader = BSHTMLLoader(file_path, open_encoding="utf-8")
docs = loader.load()

## Document Splitting

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

## Vectorstore and Embedding

### Spacy Embedding

In [20]:
# !pip install spacy
# !spacy download en_core_web_sm

In [21]:
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings

embedder = SpacyEmbeddings(model_name="en_core_web_sm")

In [22]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embedder)

### HuggingFace Embedding

In [9]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embedder = HuggingFaceInferenceAPIEmbeddings(
    api_key=hugging_api_key, model_name="Griffin88/sentence-embedding-LaBSE"
)

In [10]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embedder)


## Retrival and Generation

In [6]:
from langchain_core.prompts import PromptTemplate
from groq import Groq

class RAGPipeline:
    def __init__(self, vectorstore, api_key, model_name="gemma2-9b-it", k=6):
        self.vectorstore = vectorstore
        self.api_key = api_key
        self.model_name = model_name
        self.k = k
        self.retriever = self.vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": self.k})
        self.prompt_template = PromptTemplate.from_template(self._get_template())
        self.client = Groq(api_key=self.api_key)
        
    def _get_template(self):
        return """
        استخدم السياقات التالية للإجابة على السؤال في النهاية.
        إذا كنت لا تعرف الإجابة، فقط قل إنك لا تعرف، لا تحاول تصنيع إجابة.
        حافظ على إجابتك شاملة وصحيحة قدر الإمكان.
        دائمًا قل 'شكرًا للسؤال!' في نهاية الإجابة.
        \n السياق: {context}
        \n السؤال: {question}
        \n الإجابة المفيدة:
        """

    def generate_response(self, question):
        retrieved_docs = self._retrieve_documents(question)
        prompt = self._create_prompt(retrieved_docs, question)
        response = self._query_model(prompt)
        return response

    def _retrieve_documents(self, question):
        retrieved_docs = self.retriever.invoke(question)
        return {f'doc_{i}': doc.page_content for i, doc in enumerate(retrieved_docs)}

    def _create_prompt(self, docs, question):
        return self.prompt_template.format(context=docs, question=question)

    def _query_model(self, prompt):
        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        
        response = ""
        for chunk in completion:
            response += chunk.choices[0].delta.content or ""
        
        return response

In [7]:
rag_pipeline = RAGPipeline(vectorstore, groq_api_key)

## Tests

In [8]:
question = "بلاقي سيرياتيل كاش عتطبيق أقرب إليك؟"
print('Question: ', question)
response = rag_pipeline.generate_response(question)
print('Response: ', response)

Question:  بلاقي سيرياتيل كاش عتطبيق أقرب إليك؟
Response:  أنا ما أفهم سؤالك.  شكرًا للسؤال! 



In [29]:
questions = [
    "كيف بنزل تطبيق أقرب إليك",
    "شو هو تطبيق أقرب إليك",
    "كان الجهاز المستخدم يدعم إصدار أندرويد قديم",
    "ما هي الخدمات المتاحة لمستخدمي خطوط مسبق الدفع؟",
    "ما هي الخدمات المتاحة لمستخدمي خطوط لاحق الدفع؟",
    "كيف فيني أحول سيرياتيل كاش؟",
    "شو هو سيرياتيل كاش؟",
    "لماذا تطبيق أقرب إليك لايعمل؟",
    "ما هي سيرياتيل؟"
]

for question in questions:
    print('Question: ', question)
    response = rag_pipeline.generate_response(question)
    print('Response: ', response)

Question:  كيف بنزل تطبيق أقرب إليك
Response:  يمكنك تحميل تطبيق أقرب إليك من خلال موقع سيريتل www.syriatel.sy (تطبيقات الموبايل)، أو من خلال متجر سيريا ستور، أو عن طريق الرمز .*580#.

شكرًا للسؤال! 

Question:  شو هو تطبيق أقرب إليك
Response:  تطبيق "أقرب إليك" هو تطبيق إلكتروني يُقدم للمشتركين في شبكة سيريتل عدة خدمات, مثل:

* **إعلام العميل بالمعلومات** التي يمكن التحكم بها من خلال التطبيق

* **إدارة الخدمات** المتوفرة للزبون عبر التطبيق.

* **التحميل:** يمكن تحميل التطبيق من خلال موقع سيريتل، أو من خلال متجر سيريا ستور، أو بالضغط على الرمز *580# . 


شكرًا للسؤال! 

Question:  كان الجهاز المستخدم يدعم إصدار أندرويد قديم
Response:  عند التعامل مع حالة جهاز مزود بِإصدار قديم من Android، ينبغي على الموظف اتباع الخطوات التالية:

1. **استفسار الزبون**:  سؤال الزبون عن  إمكانية  تجربة "الشريحة" "على جهاز" يدعم إصدارًا أحدث من Android.
2. **شرح الموظف**: إذا أبدى الزبون رغبة في الاستمرار،  يجبّ على الموظف شرح  حاجة   "الجهاز" إلى إصدار أحدث  من نظام  Android  لتجربة "الشريحة" بشكل  سلس.



# Done!